In [1]:
# ! ollama run gemma:7b
# ! ollama run llama3.2

In [2]:
# Imports
import datetime
import time

from langchain_core.vectorstores import InMemoryVectorStore
from langchain_text_splitters import RecursiveCharacterTextSplitter

from langchain_community.document_loaders import UnstructuredMarkdownLoader

from langchain_community.llms import Ollama
from langchain_community.embeddings import OllamaEmbeddings

from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_core.prompts import ChatPromptTemplate


In [4]:
# Load LLM

MODEL = 'llama3.2'
# MODEL = 'gemma:7b'

llm = Ollama(model=MODEL)
embeddings = OllamaEmbeddings(model=MODEL)

In [5]:
def summarize_chapter(file_name, file_name_output, question):

  # Vars
  now = datetime.datetime.now()
  start_time = time.time()
  path_file_load = './book/'+file_name
  path_file_result = './output/'+file_name_output
  path_file_history = './output/history/'+ str(now) + '--' + file_name_output

  # Load file
  loader = UnstructuredMarkdownLoader(path_file_load)
  docs = loader.load()  
  # text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
  # splits = text_splitter.split_documents(docs)
  # vectorstore = InMemoryVectorStore.from_documents(
  #     documents=splits, embedding=OpenAIEmbeddings()
  # )

  # Create vector store
  vectorstore = InMemoryVectorStore.from_documents(
    documents=docs, embedding=embeddings
  )

  retriever = vectorstore.as_retriever()

  # Create pronpt
  system_prompt = (
    # "You are an assistant for question-answering tasks. "
    "Use the following chapter to answer the question."
    "If you don't know the answer, say that you don't know. "
    # "Use three sentences maximum and keep the "
    # "answer concise."
    "\n\n"
    "{context}"
  )

  prompt = ChatPromptTemplate.from_messages([
    ("system", system_prompt),
    ("human", "{input}"),
    ])

  # Create Pipe
  question_answer_chain = create_stuff_documents_chain(llm, prompt)
  rag_chain = create_retrieval_chain(retriever, question_answer_chain)

  # Execute
  results = rag_chain.invoke({"input": question})

  end_time = time.time()

  # Write
  f = open(path_file_result, "w")
  f.write(results['answer'])
  f.close()

  f = open(path_file_history, "w")
  f.write(f'model: {MODEL}\n')
  f.write(f'start at: {now}\n')
  f.write(f'system_prompt: "{system_prompt.join('; ')}"\n')
  f.write(f'question: {question}\n')
  f.write(f'time to process: {end_time - start_time}\n')
  f.write("\n\n")
  f.write(results['answer'])
  f.close()

In [11]:
summarize_chapter("00-FOREWORD.md", "00-FOREWORD.md", "escreva um resumo sobre o prefacio e crie topicos sobre o que foi dito")

In [9]:
summarize_chapter("01-ACCELERATE.md", "01-ACCELERATE.md", "escreva um resumo detalhado do captulo 'ACCELERATE'")

In [ ]:
summarize_chapter("02-MEASURING-PERFORMANCE.md", "02-0-MEASURING-PERFORMANCE.md", "escreva um resumo detalhado do captulo 'MEASURING PERFORMANCE'")

In [14]:
summarize_chapter("02-MEASURING-PERFORMANCE.md", "02-1-MEASURING-PERFORMANCE.md", "escreva todos os Construtores e Indicadores")

In [12]:
summarize_chapter("03-MEASURING-AND-CHANGING-CULTURE.md", "03-MEASURING-AND-CHANGING-CULTURE.md", "escreva um resumo detalhado do captulo 'MEASURING AND CHANGING CULTURE'")

In [15]:
summarize_chapter("04-TECHNICAL-PRACTICES.md", "04-TECHNICAL-PRACTICES.md", "escreva um resumo detalhado do captulo 'TECHNICAL PRACTICES'")

In [16]:
summarize_chapter("05-ARCHITECTURE.md", "05-ARCHITECTURE.md", "escreva um resumo detalhado do captulo 'ARCHITECTURE'")

In [17]:
summarize_chapter("06-INTEGRATING-INFOSEC-INTO-THE-DELIVERY-LIFECYCLE.md", "06-INTEGRATING-INFOSEC-INTO-THE-DELIVERY-LIFECYCLE.md", "escreva um resumo detalhado do captulo 'INTEGRATING INFOSEC INTO THE DELIVERY LIFECYCLE'")

In [19]:
summarize_chapter("07-MANAGEMENT-PRACTICES-FOR-SOFTWARE.md", "07-MANAGEMENT-PRACTICES-FOR-SOFTWARE.md", "escreva um resumo detalhado do captulo 'MANAGEMENT PRACTICES FOR SOFTWARE'")

In [20]:
summarize_chapter("08-PRODUCT-DEVELOPMENT.md", "08-PRODUCT-DEVELOPMENT.md", "escreva um resumo detalhado do captulo 'PRODUCT DEVELOPMENT'")